In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import os
currentdir=os.getcwd()
parentdir = os.path.dirname(currentdir)
os.sys.path.insert(1, parentdir+'/src')

In [ ]:
import numpy as np
import pandas as pd
import pinocchio as pin
from pinocchio.rpy import matrixToRpy, rpyToMatrix, rotate
import matplotlib.pyplot as plt
from mocap.viz_visualizer import VisualModel
import time
import pickle
import warnings
warnings.filterwarnings('ignore')
pin.switchToNumpyMatrix()

In [ ]:
# load humanoid model
m = VisualModel(display=True)

In [ ]:
# test quaternion

# pinocchio quaternion order: x, y, z, w
x = [0, 0, 0, 1]
y = [0,0,0]
quat_norm = pin.Quaternion(np.matrix(x).T).normalized()
R = quat_norm.toRotationMatrix()
rpy = matrixToRpy(R)
R_ = rpyToMatrix(np.matrix(y).T)
print(quat_norm, R, rpy, R_)

print(rotate('x', -np.pi / 2))

In [ ]:
def read_data(path):
    data = pd.read_csv(path, skiprows = 6)
    df = pd.DataFrame(data)
    #Data processing
    c_quat = [] #chest rotation
    s_quat = [] #shoulder rotation
    u_quat = [] #up arm rotation
    f_quat = [] #front arm rotation
    
    for i in range(0, len(data)) :
        c_quat.append([df.iloc[i][28], df.iloc[i][29], df.iloc[i][30], df.iloc[i][31]])
        s_quat.append([df.iloc[i][119], df.iloc[i][120], df.iloc[i][121], df.iloc[i][122]])
        u_quat.append([df.iloc[i][132], df.iloc[i][133], df.iloc[i][134], df.iloc[i][135]])
        f_quat.append([df.iloc[i][145], df.iloc[i][146], df.iloc[i][147], df.iloc[i][148]])
    print("data processing done")
    
    return c_quat, s_quat, u_quat, f_quat



def toRot(data):
    ret = []
    for quat in data:
        quat_norm = pin.Quaternion(np.matrix(quat).T).normalized()
        R=quat_norm.toRotationMatrix()
        ret.append(R)
    return ret

def calibrate_data(data, data_reg):
    ret = []
    for d in data:
        R=d.dot(data_reg)
        ret.append(R)
    return ret

def reMapWorldMatList(data):
    ret = []
    m = np.matrix([[1.0, 0.0, 0.0],
                    [0.0, 0.0,  1.0],
                    [0.0, -1.0,  0.0]])
    for d in data:
        R = m.dot(d)
        ret.append(R)
        
    return ret
def reMapWorldMat(data):
    m = np.matrix([[1.0, 0.0, 0.0],
                    [0.0, 0.0,  1.0],
                    [0.0, -1.0,  0.0]])
    R = m.dot(data)
        
    return R

def toRpy(data):
    ret = []
    for d in data:
        ret.append(matrixToRpy(d))
    return ret

def remap(data_child, data_mother):
    ret = []
    for dc, dm in zip(data_child, data_mother):
        R = np.linalg.inv(dm).dot(dc)
        ret.append(R)
    return ret

def plotRpy(rpy, title=""):
    plt.figure()
    xrange = np.arange(len(rpy))/240.
    plt.plot(xrange,[_rpy[0].item()/np.pi*180 for _rpy in rpy], label='x')
    plt.plot(xrange,[_rpy[1].item()/np.pi*180 for _rpy in rpy], label='y')
    plt.plot(xrange,[_rpy[2].item()/np.pi*180 for _rpy in rpy], label='z')
    plt.legend()
#     plt.ylim([-180, 180])
    plt.xlabel('time [s]')
    plt.ylabel(title+' [degree]')

In [ ]:
calibrate = True

if calibrate:
    #Use shoulder pitch to calibrate the initial posture
    #Remap the data to desired direction coordinate
    
#     path_ = 'calibration/shoulder_pitch/Take 2021-12-16 02.17.24 PM_007.csv'
    path_ = parentdir+'/data/calibration/shoulder_roll/Take 2021-12-16 02.17.24 PM_004.csv'
    c_quat_, s_quat_, u_quat_, f_quat_ = read_data(path_)
    
    c_rot_ = toRot(c_quat_)
    s_rot_ = toRot(s_quat_)
    u_rot_ = toRot(u_quat_)
    f_rot_ = toRot(f_quat_)
    
    c_rot_nw = reMapWorldMatList(c_rot_)
    s_rot_nw = reMapWorldMatList(s_rot_)
    u_rot_nw = reMapWorldMatList(u_rot_)
    f_rot_nw = reMapWorldMatList(f_rot_)

    reg_c_rot_nw = np.linalg.inv(c_rot_nw[0])
    reg_s_rot_nw = np.linalg.inv(s_rot_nw[0])
    reg_u_rot_nw = np.linalg.inv(u_rot_nw[0])
    reg_f_rot_nw = np.linalg.inv(f_rot_nw[0])
    
    pkl = []
    pkl.append(reg_c_rot_nw)
    pkl.append(reg_s_rot_nw)
    pkl.append(reg_u_rot_nw)
    pkl.append(reg_f_rot_nw)
    
    with open(parentdir+'/data/calibration/calibarate_data.pickle', 'wb') as handle:
        pickle.dump(pkl, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
else:
    with open(parentdir+'/data/calibration/calibarate_data.pickle', "rb") as handle:
        pkl = pickle.load(handle)
    reg_c_rot_nw = pkl[0]
    reg_s_rot_nw = pkl[1]
    reg_u_rot_nw = pkl[2]
    reg_f_rot_nw = pkl[3]

In [ ]:
reg_c_rot_nw, reg_s_rot_nw, reg_u_rot_nw, reg_f_rot_nw

In [ ]:
#load data from csv

# path = 'calibration/shoulder_pitch/Take 2021-12-16 02.17.24 PM_007.csv'
# path = 'calibration/shoulder_roll/Take 2021-12-16 02.17.24 PM_004.csv'
# path = 'calibration/shoulder_yaw/Take 2021-12-16 02.17.24 PM_010.csv'
# path = 'calibration/elbow/Take 2021-12-16 02.17.24 PM_013.csv'
path =  parentdir+'/data/throw/1/Take 2021-12-16 02.17.24 PM_036.csv'
c_quat, s_quat, u_quat, f_quat = read_data(path)

c_rot = toRot(c_quat)
s_rot = toRot(s_quat)
u_rot = toRot(u_quat)
f_rot = toRot(f_quat)

# map to new world frame
c_rot_nw = reMapWorldMatList(c_rot)
s_rot_nw = reMapWorldMatList(s_rot)
u_rot_nw = reMapWorldMatList(u_rot)
f_rot_nw = reMapWorldMatList(f_rot)

In [ ]:
#convert data based on calibration data
c_rot_cali = calibrate_data(c_rot_nw, reg_c_rot_nw)
s_rot_cali = calibrate_data(s_rot_nw, reg_s_rot_nw)
u_rot_cali = calibrate_data(u_rot_nw, reg_u_rot_nw)
f_rot_cali = calibrate_data(f_rot_nw, reg_f_rot_nw)

In [ ]:
um_rot = remap(u_rot_cali, s_rot_cali)
fm_rot = remap(f_rot_cali, u_rot_cali)

In [ ]:
c_rpy = toRpy(c_rot_cali)
u_rpy = toRpy(um_rot)
f_rpy = toRpy(fm_rot)

# y, x, z

# x, y, z
plotRpy(c_rpy, title="chest")
plotRpy(u_rpy, title="upperarm")
plotRpy(f_rpy, title="forearm")

In [ ]:
q = m.q0.copy()
m.viz.display(q)
traj = []
waist_traj = []

for c, u, f in zip(c_rpy, u_rpy, f_rpy):
    traj.append([-u[0].item(),u[1].item(), u[2].item(), f[0].item()])
    waist_traj.append(-1*c[2])

q[24:28] = np.matrix(traj[0]).T
q[19] = waist_traj[0]
m.viz.display(q)

In [ ]:
for w, t in zip(waist_traj, traj):
    q[19] = w
    q[24:28] = np.matrix(t).T
    m.viz.display(q)